In [16]:
%reload_ext autoreload
%autoreload 2
## sys package
import os, sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"  # specify which GPU(s) to be used
sys.path.append("./prediction_models/input/prostate-cancer-grade-assessment/")
sys.path.append("./input/")
## warning off
import warnings
warnings.filterwarnings("ignore")

## general package
import random
from tqdm import tqdm_notebook as tqdm
import numpy as np
import pandas as pd
from torch.utils.data import *

## customized package
from input.inputPipeline_spine_infer import PandaPatchDatasetInfer
from model.resnext_ssl_stiching import *

### Define test folder

In [17]:
DATA = './input/prostate-cancer-grade-assessment/test_images'
TEST = './input/prostate-cancer-grade-assessment/test.csv'
TRAIN = './input/prostate-cancer-grade-assessment/train.csv'
DATA_TRAIN = './input/prostate-cancer-grade-assessment/train_images'
SAMPLE = './input/prostate-cancer-grade-assessment/sample_submission.csv'
is_test = os.path.exists(DATA)

### Define Dataset, Dataloader Specs

In [18]:
tsfm = None

### Define Model

In [19]:
models = []
weights = [f'./train/weights/Resnext50_36patch_adam_cos_pretrain_spine/Resnext50_36patch_adam_cos_pretrain_spine_{i}_best.pth.tar' for i in [0]]
for path in weights:
    state_dict = torch.load(path)
    model = Model()
    model.load_state_dict(state_dict)
    model.float()
    model.eval()
    model.cuda()
    models.append(model)

del state_dict

Using cache found in /home/docker-user/.cache/torch/hub/facebookresearch_semi-supervised-ImageNet1K-models_master
Using cache found in /home/docker-user/.cache/torch/hub/facebookresearch_semi-supervised-ImageNet1K-models_master


### Inference

In [20]:
sub_df = pd.read_csv(SAMPLE)
## if there are data in "test_images", (only happens when you submit your notebook, do inference)

sz = 256
bs = 2

if is_test:
    dataset0 = PandaPatchDatasetInfer(TEST, DATA, sz, transform=tsfm, N = 36, rand = False)
#     dataset1 = PandaPatchDatasetInfer(TEST, DATA, sz, transform=tsfm, N = 36, rand = False, mode = 2)
else:
    dataset0 = PandaPatchDatasetInfer(TRAIN, DATA_TRAIN, sz, transform=tsfm, N = 36, rand = False)
#     dataset1 = PandaPatchDatasetInfer(TRAIN, DATA_TRAIN, sz, transform=tsfm, N = 36, rand = False, mode = 2)
dataloader0 = DataLoader(dataset0, batch_size=bs,
                        shuffle=False, num_workers=4)
# dataloader1 = DataLoader(dataset1, batch_size=bs,
#                         shuffle=False, num_workers=4)

In [21]:
names = [] ## record image names and predictions
preds = {}
for i in range(1 * len(models)):
    preds[f'{i}'] = []

In [22]:
## Model inference
with torch.no_grad():
    for idx, data in enumerate(tqdm(dataloader0)):
        img, name = data
        img = img.to(device)
        for i in range(len(models)):
            logits = models[i](img)['out']
            preds[f'{i}'].append(logits)
        names.append(name)
        if not is_test:
            if idx >= 5:
                break

#     for idx, data in enumerate(tqdm(dataloader0)):
#         img, name = data
#         img = img.to(device)
#         for i in range(len(models)):
#             logits = models[i](img)['out']
#             preds[f'{len(models) + i}'].append(logits)
#         if not is_test:
#             if idx >= 5:
#                 break
      
    names = np.concatenate(names)
    predictions = 0
    for i in range(1 * len(models)):
        predictions += torch.cat(preds[f'{i}']).sigmoid().cpu()
    predictions = predictions / (2 * len(models))
    predictions = predictions.sum(1).round().numpy().astype(int)
#     print("preds", preds.shape)
    sub_df = pd.DataFrame({'image_id': names, 'isup_grade': predictions})

sub_df.to_csv('submission.csv', index=False)
sub_df.head()

KeyError: Caught KeyError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/pandas/core/indexes/base.py", line 2646, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 111, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", line 1619, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas/_libs/hashtable_class_helper.pxi", line 1627, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'isup_grade'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/opt/conda/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 44, in f